In [1]:
import os
import json
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

import torch
from torchvision import transforms
from torch.utils.data import DataLoader

from data_generator.prop_generator import write_props_json
from blender_scripts.blender_launcher import launch_blender

from parameters import BATCH_SIZE, train_dir, val_dir, test_dir
from parameters import TRAINING_SET_SIZE, VALIDATION_SET_SIZE, TEST_SET_SIZE
from parameters import render_data_script, state_dict_path, state_dict_dir
from parameters import device
from custom_nn.custom_nn import Net, image_dataset

from custom_nn.target_loader import target_loader
from custom_nn.train import train
from custom_nn.test import test
from utils.utils import compare_images, load_from_dir, select_image_dir


In [2]:
print(os.getcwd())
skybox_list = os.listdir("blender_files\Backgrounds")
skybox_list[0]

C:\Users\liom1\Documents\projects\ML\blenderML


'city.exr'

In [3]:
skybox_list

['city.exr',
 'courtyard.exr',
 'forest.exr',
 'hayloft_4k.exr',
 'interior.exr',
 'night.exr',
 'studio.exr',
 'sunrise.exr',
 'sunset.exr']

In [3]:
for i, sky in enumerate(skybox_list):
    print(i, sky)

0 city.exr
1 courtyard.exr
2 forest.exr
3 hayloft_4k.exr
4 interior.exr
5 night.exr
6 studio.exr
7 sunrise.exr
8 sunset.exr


In [4]:


state_dict_path = ""
while True:
    try:
        state_dict_path = skybox_list[int(input("Which state dict did you want to load?"))]
        break
    except:
        print("Invalid option, please try again")
print("skybox_path set to: " + state_dict_path)        

Which state dict did you want to load?10
Invalid option, please try again
Which state dict did you want to load?l
Invalid option, please try again
Which state dict did you want to load?asdfj ia
Invalid option, please try again
Which state dict did you want to load?0
skybox_path set to: city.exr


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])


val_targets=target_loader(val_dir) 
val_data_set = image_dataset('data\\test\\renders',val_targets,transform=transform)
#subset_val = [val_data_set[i] for i in range(10)]
#print(f"{val_data_set[:10]=}")
#val_data_loader = DataLoader(subset_val, batch_size=1, shuffle=False)

In [ ]:
model = Net()
model.to(device)
model.load_state_dict(torch.load(state_dict_path))

In [ ]:
val_sub_set = val_data_set[:10]

In [ ]:
val_sub_set_loader = DataLoader(val_sub_set, batch_size=1, shuffle=False)

In [ ]:
b = [a for a in enumerate(val_sub_set_loader)]
   

In [ ]:
predictions = test(model, val_sub_set_loader)

In [ ]:
predictions

In [ ]:
write_props_json("data\\val\\pred",predictions)

In [ ]:
launch_blender(data=val_dir+"\\pred", script=render_data_script)

In [ ]:
def numpy_max_min(image):
    #image = image.cpu().numpy().transpose((1,2,0))
    return ((image - image.min()) / (image.max() - image.min()))

In [ ]:
def numpy_max_min(image):
    #image = image.cpu().numpy().transpose((1,2,0))
    return ((image - image.min()) / (image.max() - image.min()))

In [ ]:
pred_render_paths = os.listdir(val_dir+"\\pred\\renders")

In [ ]:
pred_render_imgs = []
for image_file in pred_render_paths:
    pred_render_imgs.append(mpimg.imread(os.path.join(val_dir+"\\pr=ed\\renders",image_file)))

In [ ]:
dir_list = {}
with os.scandir(val_dir+"\\pred\\renders") as dir_contents:
    for ind, entry in enumerate(dir_contents):
        if entry.is_dir():
            dir_list.update({ind : entry.name})
selection = int(input(f'Which dir would you like to chose? {dir_list}'))
selected_dir = dir_list[selection]
selected_dir

In [ ]:
val_sub_set[0][0].cpu()

In [ ]:
fig, axes = plt.subplots(len(pred_render_imgs), 2, figsize=(10, 2 * len(pred_render_imgs)))

for i in range(len(pred_render_imgs)):
    axes[i, 0].imshow(numpy_max_min_tf(val_sub_set[i][0]), aspect='equal')
    axes[i, 0].set_title("X")
    axes[i, 0].axis("off")

    axes[i, 1].imshow(numpy_max_min(pred_render_imgs[i]), aspect='equal')
    axes[i, 1].set_title("Y")
    axes[i, 1].axis("off")

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

#train_targets = target_loader(train_dir)
#train_data_set = image_dataset('data\\train\\renders',train_targets,transform=transform)
#train_data_loader = DataLoader(train_data_set, batch_size=BATCH_SIZE, shuffle=True)

val_targets=target_loader(val_dir) 
val_data_set = image_dataset('data\\test\\renders',val_targets,transform=transform)
#val_data_loader = DataLoader(val_data_set, batch_size=1, shuffle=False)

# load neural net, define loss and optimizer

model = Net()
model.to(device)
model.load_state_dict(torch.load(state_dict_path))


# train nn on train props

#train(model, data_loader=train_data_loader)

# test nn on test props
val_sub_set = val_data_set[:10]
val_sub_set_loader = DataLoader(val_sub_set, batch_size=1, shuffle=False)

val_sub_set_predictions = test(model, val_sub_set_loader)
write_props_json("data\\val\\pred",val_sub_set_predictions)
launch_blender(data=val_dir+"\\pred", script=render_data_script)

pred_render_dir = select_image_dir(val_dir+"\\pred\\renders")
#pred_render_paths = os.listdir(render_dir)


In [ ]:
print(f"{pred_render_dir}")

In [ ]:
pred_render_images = load_from_dir(pred_render_dir)

In [ ]:
compare_images(pred_render_images,val_sub_set)
